In [1]:
%load_ext autoreload
%autoreload 2

import sys
import numpy as np

from scipy import stats

sys.path.append('/home/hpham/code')

# Load the config.yaml to get all the directories
from waterchange.config import read_config
from waterchange.datasets import setup_directory
config = read_config()
input_data = setup_directory(config)

Loading /home/hpham/code/waterchange/config/../config.yaml.
Config loaded successfully.
Getting sorted paths to pixel data points.
Processing folder: /home/hpham/data/geo/h016v012/TSFitLine
Folder pattern: record_change_r*.mat
Sort key: [-9, -4]
Found 2000 files
First and last directory in this folder                      
 /home/hpham/data/geo/h016v012/TSFitLine/record_change_r00001.mat                      
 /home/hpham/data/geo/h016v012/TSFitLine/record_change_r02000.mat
Processing folder: /home/hpham/data/geo/h016v012/TSFitLine_1
Folder pattern: record_change_r*.mat
Sort key: [-9, -4]
Found 3000 files
First and last directory in this folder                      
 /home/hpham/data/geo/h016v012/TSFitLine_1/record_change_r02001.mat                      
 /home/hpham/data/geo/h016v012/TSFitLine_1/record_change_r05000.mat
Found 5000 in COMBINED folder.
First and last directory in COMBINED folders                 
 /home/hpham/data/geo/h016v012/TSFitLine/record_change_r00001.mat         

In [10]:
from waterchange.utils import get_logger
from waterchange.models import predict_average_water_change_at_break_point
get_logger("INFO")

res_train = []
res_infer = []

for index in input_data.rois:
    infer_region_id = index
    train_region_id = [x for x in input_data.rois if x != infer_region_id]
    _, acc_train, acc_infer  = predict_average_water_change_at_break_point(
    input_data,
    sub_dir_in_train="water_coefs_break_point",
    train_region_id=train_region_id,
    infer_region_id=infer_region_id)
    res_train.append(acc_train)
    res_infer.append(acc_infer)


Training:
Overall Accuracy: 0.8636796949475691 - [0.9421538461538461, 0.6595092024539877, 0.4489795918367347]
Inference
Overall Accuracy: 0.46380846325167036 - [0.953083109919571, 0.3081761006289308, 0.03278688524590164]
Training:
Overall Accuracy: 0.8195991091314031 - [0.843915343915344, 0.7315175097276264, 0.8323353293413174]
Inference
Overall Accuracy: 0.7452714320805699 - [0.9248305606900801, 0.03951367781155015, 0.03592814371257485]
Training:
Overall Accuracy: 0.8268292682926829 - [0.9083832335329342, 0.632420091324201, 0.6818181818181818]
Inference
Overall Accuracy: 0.6518624641833811 - [0.9558139534883721, 0.33884297520661155, 0.02040816326530612]
Training:
Overall Accuracy: 0.8231511254019293 - [0.9126040428061831, 0.6161369193154034, 0.6574307304785895]
Inference
Overall Accuracy: 0.6867862969004894 - [0.9809264305177112, 0.2760416666666667, 0.14814814814814814]
Training:
Overall Accuracy: 0.8342989571263036 - [0.9209919261822376, 0.6238938053097345, 0.6972704714640199]
Infere

In [12]:
from waterchange.models import acc_stat, cal_p_value
acc_stat(res_train)
acc_stat(res_infer)

Overall: 0.83 ± 0.01
Class 0: 0.91 ± 0.03
Class 1: 0.65 ± 0.04
Class 2: 0.67 ± 0.11
Overall: 0.69 ± 0.12
Class 0: 0.96 ± 0.02
Class 1: 0.20 ± 0.13
Class 2: 0.11 ± 0.12


(array([0.6913336 , 0.95514131, 0.20132016, 0.10558422]),
 (array([ 0.56772684,  0.93221997,  0.07608603, -0.00996351]),
  array([0.81494036, 0.97806266, 0.32655428, 0.22113195])))

In [ ]:
"""
With scaler
Overall: 0.83 ± 0.01
Class 0: 0.91 ± 0.03
Class 1: 0.65 ± 0.04
Class 2: 0.67 ± 0.11
Overall: 0.69 ± 0.12
Class 0: 0.96 ± 0.02
Class 1: 0.20 ± 0.13
Class 2: 0.11 ± 0.12
"""